# Machine Learning Project 2 : Multi-object detection and tracking

## Installation

First, copy the needed files from github :


In [ ]:
!git clone https://github.com/clempetit/ML-project-2
!mv /content/ML-project-2/* /content/
!rm -r ML-project-2/

Then some additional packages need to be installed :

In [ ]:
%cd /content/tensorflow_models/research

!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!pip install .
!pip install opencv-contrib-python -U

Test Installation (still in tensorflow_models/research) :

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

Test GPU support :

In [ ]:
!python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

We also define some useful paths to make the following commands lighter.

In [ ]:
%cd /content
TRAINING_PATH = 'training'
IMAGE_PATH = 'images'
TRAINED_MODEL_PATH = TRAINING_PATH+'/trained-model'
PRETRAINED_MODEL_PATH = TRAINING_PATH+'/pre-trained-models'
TF_RECORD_PATH = TRAINING_PATH+'/TFRecords'
LABEL_MAP_PATH = TF_RECORD_PATH + '/label_map.pbtxt'
RESULT_PATH = 'results/'

/content


## Training

IMPORTANT : We already provided the backup of a training (see inside the `training/trained-model` folder). If you choose to redo the training, please first delete all the files inside this `training/trained-model` folder. Otherwise, you can skip this section, and go directly to the "Demo" section.

Create the TFRecords, containing images and detection data for the training :

In [ ]:
!python create_tfrecord.py {IMAGE_PATH + '/train'} {TF_RECORD_PATH + '/train.record'} {TF_RECORD_PATH + '/label_map.pbtxt'}
!python create_tfrecord.py {IMAGE_PATH + '/test'} {TF_RECORD_PATH + '/test.record'} {TF_RECORD_PATH + '/label_map.pbtxt'}

Modify the model config for our case :

In [ ]:
!python create_config.py {PRETRAINED_MODEL_PATH + "/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"} {LABEL_MAP_PATH} {TF_RECORD_PATH} {TRAINED_MODEL_PATH} -b 8

Train the model (expected to take around 30 minutes to complete with batch size set to 8 in the pipeline config, and with 5000 steps) :

In [ ]:
!python tensorflow_models/research/object_detection/model_main_tf2.py --model_dir={TRAINED_MODEL_PATH} --pipeline_config_path={TRAINED_MODEL_PATH + '/pipeline.config'} --num_train_steps=5000

## Demo

Run the demo with our SSD model and MOSSE tracking. Here the detection is performed once every 10 frames by default (frequency can be changed with -dt flag) for faster computation. Other frames rely on tracking.

In [ ]:
!rm -r results; mkdir results
!python demo_ssd_mosse.py People.mp4 {TRAINED_MODEL_PATH} -o {RESULT_PATH}

Save the results in a zip file

In [ ]:
!zip -r results.zip {RESULTS_PATH}

Run the demo with our SSD model and DeepSORT tracking. This tracker allows to accurately keep track of specific objects but detection needs to be performed at every frame, so therefore computation is slower than the first demo as here.

In [ ]:
!rm -r results; mkdir results
!python demo_ssd_deepsort.py People.mp4 {TRAINED_MODEL_PATH} deep_sort/mars-small128.pb -o {RESULT_PATH}

In [ ]:
!zip -r results.zip {RESULTS_PATH}

In addition, you can also have a look at a YOLO v3 demo (with KCF tracking), that we had previously written in order to have a first glimpse at object detection, and that we used later to make comparisons with our own SSD model. In order to run this script, you will need first to download the weights [yolov3-spp.weights](https://pjreddie.com/media/files/yolov3-spp.weights) from pjreddie.com (the file weighs about 200MB, which was too big to put inside the github repo) and put the file inside the `yolov3-files` folder (to upload a file, click on the three dots visible when you hover above the directory, and then select "upload". The file weighs around 200MB, therefore the upload may take long, sorry for the inconvenience) :

In [ ]:
!rm -r results; mkdir results
!python demo_yolo_kcf.py People.mp4 -o {RESULT_PATH}